In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn.linear_model  as sk  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score

import pandas as pd
from helper import *
import tensorflow as tf

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.regularizers import L1L2

Using TensorFlow backend.


# Stuff that needs to be done

- Find another framework.
    - Sklearn
    - Spark
    - Tensorflow/R
- Algorithms
    - Linear Regression
    - Logistic Regression
    - Maybe SVM /Random forest
- Find two metrics 
    - RMSE 
    - Some other
- Do 70/30 split training and 10 cross validation
    - only did cross validation
- Find another dataset 
    - Sum data with noise 
    -  Maybe housing data set
    

## Import DataSet

In [2]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [3]:
get_data(all_urls) # retrieves the data if there is no data folder

In [4]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/housing dataset.csv" # has more than 30 features
# need one more
# what a brilliant idea to name files with space

In [5]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

## Load datasets sum_noise

In [12]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise.head(n=1
                  )

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array

# MADE A CHANGE HERE


In [15]:

le = preprocessing.LabelEncoder()

# Remove 'Instance' as it simply represents the row number
sumdata_noise.drop('Instance', axis = 1)

# Extract 'Noisy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise[['Noisy Target']]

# Extract 'Noisy Target Class' as regression target 

sumdata_noise_classif_Y ['Noisy Target Class'] = le.fit_transform( sumdata_noise ['Noisy Target Class'].astype('str')) 

# Extract rest columns as explananatory variables
sumdata_noise_X = sumdata_noise.iloc[:, 1:-2]

### Logistic Regression using Keras

In [25]:
#keras log reg
def keras1DLogReg(X_train, X_test, y_train, y_test):
    
    from sklearn import preprocessing
    encoder = preprocessing.LabelEncoder()
    
    #Reshape and normalise inputs
    X_train = X_train.as_matrix()
    X_test = X_test.as_matrix()   
    
    """
        
    """
    # encode class values as integers"
    encoder.fit(y_train) 
    encoded_y_train = encoder.transform(y_train)
    encoder.fit(y_test)
    encoded_y_test = encoder.transform(y_test)
    
    # convert integers to dummy variables (i.e. one hot encoded)
    y_train = np_utils.to_categorical(encoded_y_train)
    y_test = np_utils.to_categorical(encoded_y_test)
    
    
    #Build the model
    output_dim = y_train.shape[1]
    model = Sequential()
    model.add(Dense(output_dim, input_dim = X_train.shape[1], activation='softmax'))
    batch_size = 128
    epochs = 20

    #Compile ze model
    model.compile(optimizer = 'sgd', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 'precisin'])
    history = model.fit(X_train, 
                        y_train, 
                        batch_size = batch_size,
                        epochs = epochs,
                        verbose = 1,
                        validation_data=(X_test, y_test))
    score = model.evaluate(X_test, y_test, verbose = 0)

    print ('Test score: ', score[0])
    print ('Test accuracy', score[1])
     
 
    return (rmse,mae)

In [26]:
def model_cross_valdiation( X, y, dataset_name, algorithm, kfolds,regression=False): 
    
    kf = KFold(n_splits=kfolds)
    rmse = np.zeros((kfolds,1))
    mae = np.zeros((kfolds,1))
    accuracy = np.zeros((kfolds,1))
    precision = np.zeros((kfolds,1))
    
    algorithm_n = algorithm.__name__ 
 

    for (i, (train_index, test_index)) in enumerate(kf.split(X)):


        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        

        # accumulating errors for each fold 
        # calculates mean across rows
        if regression:     
            rmse[i], mae[i] = algorithm( X_train, X_test, y_train, y_test)   # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
           
        else: 
            accuracy[i],precision[i] = algorithm( X_train, X_test, y_train, y_test)
           
    if regression:     
        print ("Dataset: {}\nRMSE: {}\nMAE: {}\n".format( dataset_name, rmse.mean(), mae.mean()))
    else:
        print ("Dataset: {}\Accuracy: {}\nPrecision: {}\n".format(dataset_name, accuracy.mean(), precision.mean()))

In [27]:
model_cross_valdiation( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", keras1DLogReg, 10)

Train on 871321 samples, validate on 96814 samples
Epoch 1/20
871321/871321 [==============================] - 6s - loss: 13.0667 - acc: 0.1893 - val_loss: 13.0724 - val_acc: 0.1890
Epoch 2/20
871321/871321 [==============================] - 6s - loss: 13.0667 - acc: 0.1893 - val_loss: 13.0724 - val_acc: 0.1890
Epoch 3/20
871321/871321 [==============================] - 6s - loss: 13.0667 - acc: 0.1893 - val_loss: 13.0724 - val_acc: 0.1890
Epoch 4/20
871321/871321 [==============================] - 6s - loss: 13.0667 - acc: 0.1893 - val_loss: 13.0724 - val_acc: 0.1890
Epoch 5/20
871321/871321 [==============================] - 6s - loss: 13.0667 - acc: 0.1893 - val_loss: 13.0724 - val_acc: 0.1890
Epoch 6/20
871321/871321 [==============================] - 6s - loss: 13.0667 - acc: 0.1893 - val_loss: 13.0724 - val_acc: 0.1890
Epoch 7/20
871321/871321 [==============================] - 6s - loss: 13.0667 - acc: 0.1893 - val_loss: 13.0724 - val_acc: 0.1890
Epoch 8/20
871321/871321 [======

AttributeError: module 'sklearn.linear_model' has no attribute 'metrics'

In [ ]:
def model_test_split( X, y, dataset_name, algorithm, regression=False): 
    algorithm_n = algorithm.__name__
    rmse = 0
    mae = 0 
    accuracy = 0
    precision = 0  

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    if regression:     
        rmse, mae = algorithm( X_train, X_test, y_train, y_test)   # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
        print ("Algorithm: {} Dataset: {}\nRMSE: {}\nMAE: {}\n".format( 
                                                                algorithm_n,
                                                                dataset_name,
                                                                rmse,
                                                                mae))
        
    else: 
        accuracy ,precision  = algorithm( X_train, X_test, y_train, y_test)
        # accumulating errors for each fold
        print ("Algorithm: {} Dataset: {}\Accuracy: {}\nPrecision: {}\n".format(
                                                                algorithm_n,
                                                                dataset_name,
                                                                accuracy,
                                                                precision))

In [ ]:
model_test_split( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", keras1DLogReg)

In [99]:
#Logistic Regression using MNIST dataset
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
keras1DLogReg(X_train, y_train, X_test, Y_test, 784)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 2s - loss: 1.2964 - acc: 0.6874 - val_loss: 0.8180 - val_acc: 0.8289
Epoch 2/20
60000/60000 [==============================] - 1s - loss: 0.7190 - acc: 0.8393 - val_loss: 0.6106 - val_acc: 0.8625
Epoch 3/20
60000/60000 [==============================] - 1s - loss: 0.5888 - acc: 0.8581 - val_loss: 0.5274 - val_acc: 0.8742
Epoch 4/20
60000/60000 [==============================] - 1s - loss: 0.5264 - acc: 0.8686 - val_loss: 0.4811 - val_acc: 0.8805
Epoch 5/20
60000/60000 [==============================] - 1s - loss: 0.4885 - acc: 0.8751 - val_loss: 0.4510 - val_acc: 0.8854
Epoch 6/20
60000/60000 [==============================] - 1s - loss: 0.4624 - acc: 0.8791 - val_loss: 0.4293 - val_acc: 0.8890
Epoch 7/20
60000/60000 [==============================] - 1s - loss: 0.4431 - acc: 0.8831 - val_loss: 0.4130 - val_acc: 0.8926
Epoch 8/20
60000/60000 [==============================] - 2s 